In [158]:
from dask_jobqueue import SLURMCluster


cluster = SLURMCluster(
    job_name="Climt1",          # --job-name
    cores=48,                   # 24 cores per node
    processes=4,                # One process per task
    memory="400GB",             # --mem
    walltime="04:00:00",        # --time
    queue="med",                # --partition
    log_directory=".",          # Logs will be saved to the current directory
)

/home1/nalex2023/.local/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42795 instead
  warnings.warn(


In [160]:
from dask.distributed import Client


cluster.scale(jobs=10)
client = Client(cluster)
client

<Client: 'tcp://10.42.239.61:34701' processes=8 threads=96, memory=745.04 GiB>

In [165]:
client.close()
cluster.close()

In [161]:
import glob 
import pandas as pd
import xarray as xr
import sys
import os 
sys.path.append('/home1/nalex2023/main/LSB_main/Modules/LSB_prepro')
import load_arm_files_main

In [162]:
#loc = glob.glob('/home2/nalex2023/Datasets/met_manus/*.cdf')
loc = '/home2/nalex2023/Datasets/met_manus/'

files = glob.glob(loc+'/*.cdf')

dft = pd.DataFrame(files,columns=['filepath'])

dft['filename'] = dft['filepath'].str.split(os.sep).str[-1]

dft['datetime'] = pd.to_datetime(dft['filename'].str.split(
            '.',
            expand=True).iloc[:, 2] + dft['filename'].str.split(
                '.',  expand=True).iloc[:, 3],format='%Y%m%d%H%M%S')

dft_sorted = dft.sort_values('datetime').set_index('datetime',drop=True)

dft_sorted.loc['2000']


,filepath,filename
datetime,,
2000-01-01 00:00:00,/home2/nalex2023/Datasets/met_manus/twpmetC1.b...,twpmetC1.b1.20000101.000000.cdf
2000-01-02 00:00:01,/home2/nalex2023/Datasets/met_manus/twpmetC1.b...,twpmetC1.b1.20000102.000001.cdf
2000-01-03 00:00:00,/home2/nalex2023/Datasets/met_manus/twpmetC1.b...,twpmetC1.b1.20000103.000000.cdf
2000-01-04 00:00:01,/home2/nalex2023/Datasets/met_manus/twpmetC1.b...,twpmetC1.b1.20000104.000001.cdf
2000-01-05 00:00:00,/home2/nalex2023/Datasets/met_manus/twpmetC1.b...,twpmetC1.b1.20000105.000000.cdf
...,...,...
2000-12-27 00:00:01,/home2/nalex2023/Datasets/met_manus/twpmetC1.b...,twpmetC1.b1.20001227.000001.cdf
2000-12-28 00:00:01,/home2/nalex2023/Datasets/met_manus/twpmetC1.b...,twpmetC1.b1.20001228.000001.cdf
2000-12-29 00:00:01,/home2/nalex2023/Datasets/met_manus/twpmetC1.b...,twpmetC1.b1.20001229.000001.cdf


In [164]:
import numpy as np

years = pd.unique(dft_sorted.index.year)

years = [2002]  # manually set

years

months = list(map(str,np.arange(1,13)))

months = [x.zfill(2) for x in months]

outfol = '/home2/nalex2023/Datasets/LSB_OUTS/met_manus_30min/'


def preprocess(ds):
    ds_new = ds[['atmos_pressure','temp_mean','rh_mean','wspd_arith_mean','wspd_vec_mean','wdir_vec_mean','org_precip_rate_mean']]
    return ds_new

for ii in years:
    for jj in months:
        comb_str = f'{ii}-{jj}'
        try:
            dft_filtered = dft_sorted.loc[comb_str]
        except KeyError:
            continue
        dset_all = xr.open_mfdataset(dft_filtered['filepath'],chunks=None,parallel=False).chunk({'time':-1})

        dset_sub = preprocess(dset_all)
        
        dset_res = dset_sub.resample(time='30min').mean()
        
        computed = dset_res.compute()
        out_file = f'{outfol}TWP_manus_{comb_str}.nc'
        computed.to_netcdf(out_file)
        dset_all.close()
        computed.close()


/home1/nalex2023/.local/lib/python3.10/site-packages/distributed/client.py:3363: UserWarning: Sending large graph of size 13.43 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home1/nalex2023/.local/lib/python3.10/site-packages/distributed/client.py:3363: UserWarning: Sending large graph of size 11.70 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home1/nalex2023/.local/lib/python3.10/site-packages/distributed/client.py:3363: UserWarning: Sending large graph of size 13.50 MiB.
This may cause some slowdown.
C